In [16]:
data_path = 'data_raw/'
parsed_path = 'data_scrapped/'
!pip install selenium pandas  lxml tqdm bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
     |████████████████████████████████| 115 kB 1.3 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=a51b9a581dcdf84d1068507ce3e8def87ad641eeed6f8d9190174d49256f4d40
  Stored in directory: /home/mateusz-destroyer/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4


In [2]:
import codecs
from tqdm import tqdm
def save_file(save_path:str, content:str) -> None:
    """Generic function to save files"""
    with codecs.open(save_path, 'w') as file:
        file.write(content)
        
def open_file(path: str) -> bytes:
    with codecs.open(path, 'r') as file:
        content = file.read()
    return content

## Crawl the list of the IEOs

In [18]:
from selenium import webdriver
import os

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
list_ieos = 'https://icobench.com/ieo'
driver = webdriver.Remote('http://172.17.0.1:4444/wd/hub',DesiredCapabilities.CHROME)
driver.get(list_ieos)
path_to_save = os.path.join(data_path,'ieo_list_raw_html/raw_list.html')
print(path_to_save)
save_file(path_to_save,driver.page_source)
driver.close()

data_raw/ieo_list_raw_html/raw_list.html


## Scrape the list of the Ieos

In [19]:
#//*[@id="ieocols"]/div/div/table
def get_links_to_proj(root):
    all_links = "//a[@class='name notranslate']/@href"
    get_all = root.xpath(all_links)
    print(get_all)
    return get_all
    

from lxml import html
import pandas as pd
path_to_open = os.path.join(data_path,'ieo_list_raw_html/raw_list.html')
content = open_file(path_to_open)
root = html.fromstring(content)
table_content = html.tostring(root.xpath('//*[@id="ieocols"]/div/div/table')[0])
df = pd.read_html(table_content)[0]
df['links'] = get_links_to_proj(root)
df = df.drop(['Review','Project'],axis=1)
df= df.rename(columns={"Project.1": "Project_Name"})
save_path = os.path.join(parsed_path,'scrapped_list.csv')
df.to_csv(save_path,index=False)
df

['/ico/bitfinex', '/ico/tron-game-global', '/ico/perlin', '/ico/percival', '/ico/bread', '/ico/socialgood', '/ico/chars', '/ico/gifto', '/ico/windhan-energy', '/ico/ioex', '/ico/harmony', '/ico/multivac', '/ico/coti', '/ico/akropolis', '/ico/ecomi', '/ico/band-protocol', '/ico/squeezer', '/ico/azbit', '/ico/menapay', '/ico/1irstcoin', '/ico/adab-solutions', '/ico/bittorrent', '/ico/veriblock', '/ico/connect-coin', '/ico/terragreen', '/ico/dexage', '/ico/fetch-ai', '/ico/venusenergy', '/ico/redfox-labs', '/ico/res', '/ico/meca-coin', '/ico/gath3r', '/ico/aerum', '/ico/emx', '/ico/ampleforth', '/ico/matic-network', '/ico/neutro', '/ico/vecap', '/ico/spin-protocol', '/ico/verasity', '/ico/celer-network', '/ico/zerobank', '/ico/cryptoads', '/ico/coinshare', '/ico/loltoken-by-eiu-ac', '/ico/sessia', '/ico/dexon', '/ico/aizeus', '/ico/bit-agro', '/ico/zeux', '/ico/pukkamex', '/ico/auditchain', '/ico/poseidon-foundation', '/ico/gexan', '/ico/esax', '/ico/cybr', '/ico/stonecoin', '/ico/izichai

,Project_Name,Progress,Launchpad,Rating,Token,Start,links
0,BITFINEX,Ended$1000M,Bitfinex Launchpad,3.8,LEO,"05 May, 2019",/ico/bitfinex
1,Tron Game Global,Ended$80M,Multiple,2.6,GTP,"13 Jun, 2019",/ico/tron-game-global
2,Perlin,Ended$53M,Binance Launchpad,3.5,PERL,"24 Aug, 2019",/ico/perlin
3,Percival,Ended$35M,Coineal Launchpad,2.3,XPV,"05 Mar, 2019",/ico/percival
4,Bread,Ended$32M,Binance Launchpad,3.4,BRD,"16 Dec, 2017",/ico/bread
...,...,...,...,...,...,...,...
290,Level01,Ended—,p2pb2b Launchpad,4.3,LVX,"01 Jul, 2018",/ico/lvx
291,Ubecoin,Ended—,Latoken Launchpad,3.8,UBE,"12 Apr, 2018",/ico/ubecoin
292,INTELLIGENT HERO,Ended—,Exmarkets Launchpad,3.2,Intelligent Hero,"10 Jan, 2010",/ico/intelligent-hero
293,Rowan Energy Blockchain,22%,Latoken Launchpad,4.9,RWN,"01 Jul, 2020",/ico/rowan-energy-blockchain


## Crawl The Projects' websites

In [5]:
import time
import urllib.parse
from selenium.webdriver.common.keys import Keys

def create_url(extended_path,base_path='https://icobench.com/'):
    url = urllib.parse.urljoin(base_path, extended_path)
    return url

def scroll_to_the_end_html(driver) -> None:
    """Webdriver is scrolling to the end of the webpage and renders the content for the
    further scrapping."""
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.END)

driver = webdriver.Remote('http://172.17.0.1:4444/wd/hub',DesiredCapabilities.CHROME)
df_scrape_path = os.path.join(parsed_path,'scrapped_list.csv')
df_scrape = pd.read_csv(df_scrape_path)
df_scrape['links'].apply(lambda x: create_url(x))
for i,row in tqdm(df_scrape.iterrows()):
    url_scrape = create_url(row['links'])
    driver.get(url_scrape)
    time.sleep(1)
    scroll_to_the_end_html(driver)
    page_source = driver.page_source
    if i % 10 == 0: 
        time.sleep(1)
    
    path_to_save = os.path.join(data_path,f'ieo_infos_raw_html/{row["Project_Name"]}.html')
   
    save_file(path_to_save,page_source)
    
driver.close()

295it [09:12,  1.87s/it]


In [6]:
df_scrape

,Project_Name,Progress,Launchpad,Rating,Token,Start,links
0,BITFINEX,Ended$1000M,Bitfinex Launchpad,3.8,LEO,"05 May, 2019",/ico/bitfinex
1,Tron Game Global,Ended$80M,Multiple,2.6,GTP,"13 Jun, 2019",/ico/tron-game-global
2,Perlin,Ended$53M,Binance Launchpad,3.5,PERL,"24 Aug, 2019",/ico/perlin
3,Percival,Ended$35M,Coineal Launchpad,2.3,XPV,"05 Mar, 2019",/ico/percival
4,Bread,Ended$32M,Binance Launchpad,3.4,BRD,"16 Dec, 2017",/ico/bread
...,...,...,...,...,...,...,...
290,Level01,Ended—,p2pb2b Launchpad,4.3,LVX,"01 Jul, 2018",/ico/lvx
291,Ubecoin,Ended—,Latoken Launchpad,3.8,UBE,"12 Apr, 2018",/ico/ubecoin
292,INTELLIGENT HERO,Ended—,Exmarkets Launchpad,3.2,Intelligent Hero,"10 Jan, 2010",/ico/intelligent-hero
293,Rowan Energy Blockchain,22%,Latoken Launchpad,4.9,RWN,"01 Jul, 2020",/ico/rowan-energy-blockchain


## Scrape Infos |

In [10]:
import glob 
from lxml import html
def open_files():
    path = 'data_raw/ieo_infos_raw_html/*'
    for file_path in glob.glob(path):
        content = open_file(file_path)
        name = os.path.basename(file_path).split('.')[0]
        yield content,name

def scrape_information(content,name):
    root = html.fromstring(content)
    url_whitepaper_xpath = '//*[@id="whitepaper"]/p[1]/object/@data'
    try:
        url_whitepaper = root.xpath(url_whitepaper_xpath)[0]
    except:
        url_whitepaper=None
    
    return [url_whitepaper,name]
    

columns= ['whitepaper_url','Project_name']
frames =[]
for html_content,name in open_files():
    data = scrape_information(html_content,name)
    frames.append(data)
    
df = pd.DataFrame(data=frames,columns=columns)
save_path = os.path.join(parsed_path,'scrapped_ieo_infos.csv')
df.to_csv(save_path,index=False)

In [11]:
df

,whitepaper_url,Project_name
0,https://cryptococktailcoin.io/wp-content/uploa...,CryptoCocktail Coin
1,https://assets.ctfassets.net/sdlntm3tthp6/4g5p...,BNC Token
2,https://mydogdata.com/pdf/DogdataLitewhitepape...,DogData
3,https://images.fxempire.com/ico/whitepaper/dus...,Dusk Network
4,https://mecacoin.io/whitepaper/MECACoin%20Whit...,MECA Coin
...,...,...
290,https://www.youtube.com/embed/165SSYueCJw,1irstcoin
291,https://docs.google.com/presentation/d/1vqjfCw...,Ziktalk
292,https://www.ampleforth.org/paper/,Ampleforth
293,https://drive.google.com/file/d/1c8LZjH9MwkM48...,ECOMI


## Crawl PDFs

In [13]:
import requests
path_to_ieo_infos = os.path.join(parsed_path,'scrapped_ieo_infos.csv')
df = pd.read_csv(path_to_ieo_infos)
def download_pdf(url,chunk_size=3000):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = row['whitepaper_url']
   
    save_point = 'data_raw/ieo_whitepapers'
    path_to_save = os.path.join(save_point,f'{row["Project_name"]}.pdf')
    try:
        s = requests.get(url, headers=headers,stream=True)

        with open(path_to_save, 'wb') as fd:
            for chunk in s.iter_content(chunk_size):
                fd.write(chunk)
        return 'SUCCESS'
    except:
        return 'FAILED'
    
statuses = []
cols=['Project_name','whitepaper_url','DOWNLOAD_STATUS']
for i,row in tqdm(df.iterrows()):
  
    status = download_pdf(row)
    statuses.append(status)
df['DOWNLOAD_STATUS']=statuses
cols=['Project_name','whitepaper_url','DOWNLOAD_STATUS']
path_to_ieo_infos = os.path.join(parsed_path,'scrapped_whitepapers_statuses.csv')
df[cols].to_csv(path_to_ieo_infos,index=False)

295it [15:08,  3.08s/it]


In [15]:
df['DOWNLOAD_STATUS'].value_counts()

SUCCESS    233
FAILED      62
Name: DOWNLOAD_STATUS, dtype: int64

In [ ]:
df